# Code for printing the csv file with title and abstract


In [ ]:
import csv
import pandas as pd
import json
import numpy as np
import os
from IPython.display import clear_output

ultimate_dict ={}

def getvalues(PATH) :
    
    filenames = os.listdir(PATH)
    ultimate_dict = {}
    foldernames = os.listdir(PATH)

    myFields = ['Altmetric_ID', 'Altmetric_Score', 'Title', 'Abstract', 'Scopus']
    df = pd.DataFrame(columns = ['Altmetric_ID', 'Altmetric_Score', 'Title', 'Abstract', 'Scopus'])

    count = 0
    for folder in foldernames:
        filenames = os.listdir(PATH + folder)
        for file in filenames:
            
            with open(PATH + folder + '\\' +file,encoding="ISO-8859-1") as f:
                data1 = {}
                for line in f:
                    data1 = json.loads(line)
                    break
                    #print(data1)
                    
                try:

                    ultimate_dict['Altmetric_ID']=(data1['altmetric_id'])
                except KeyError:
                    ultimate_dict['Altmetric_ID']= np.NaN
                    break


                try:
                    ultimate_dict['Altmetric_Score']=(data1["altmetric_score"]["score"])
                except KeyError:
                    ultimate_dict['Altmetric_Score']=np.NaN


                try:
                    ultimate_dict['Title']=(data1['citation']['title'])
                except KeyError:
                    ultimate_dict['Title']=np.NaN


                try:
                    ultimate_dict['Abstract']= (data1['citation']['abstract'])

                except KeyError:
                    ultimate_dict['Abstract']=np.NaN



                try:
                    ultimate_dict['Scopus']=" ".join(data1['citation']['scopus_subjects'])
                except KeyError:
                    ultimate_dict['Scopus']=np.NaN



                df.loc[-1] = ultimate_dict  # adding a row
                df.index = df.index + 1  # shifting index
                df = df.sort_index()
                df = df.dropna(axis=0, how='any')
                print(count)
                count = count +1
                clear_output(wait=True)

    clean_df = df.dropna(axis=0,how = 'any')
    clean_df[myFields].to_csv('C:\\Users\\Devesh\\Documents\\data\\extractone\\jun-10th-dump.tar.gz.part\\keys\\key.csv', index = None, header = True)


PATH = 'C:\\Users\\Devesh\\Documents\\data\\extractone\\jun-10th-dump.tar.gz.part\\keys\\' 
getvalues(PATH) 


## Code for comparing title and abstracts 


In [ ]:
import re
import pandas as pd
from nltk.tokenize import word_tokenize
from nltk.tokenize import regexp_tokenize
import nltk
from matplotlib import pyplot as plt
from collections import Counter
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
from gensim.corpora.dictionary import Dictionary
from gensim.models.tfidfmodel import TfidfModel
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.feature_extraction.text import TfidfVectorizer
from math import sqrt, log
from collections import defaultdict
from itertools import chain, product
import numpy as np
from IPython.display import clear_output

nltk.download('averaged_perceptron_tagger')
nltk.download('punkt')
nltk.download('stopwords')
nltk.download('wordnet')

#also need to download gensim on the command prompt, for (windows,anaconda) it is conda install gensim

# function to split the title and abstract into tokens, control moves to stoken_clean
def split_tokenize(title_string, abstract_string ):

    title_sentences = nltk.sent_tokenize(title_string)
    abstract_sentences = nltk.sent_tokenize(abstract_string)
    
    title_tokens = [word_tokenize(sent) for sent in title_sentences]
    abstract_tokens = [word_tokenize(sent) for sent in abstract_sentences]


    return(title_tokens,abstract_tokens)
    
    
#this function converts words to lower case, removes stopwords, output goes for bagging
def token_clean(title_tokens, abstract_tokens): 
    
    title_tokens_loweralpha = [it.lower() for it in title_tokens
                              if(it not in stopwords.words('english'))]
    abstract_tokens_loweralpha = [it.lower() for it in abstract_tokens
                                  if(it not in stopwords.words('english'))]
    
#    return title_tokens_loweralpha,abstract_tokens_loweralpha
    WDL = WordNetLemmatizer()
    
    title_clean = [WDL.lemmatize(it) for it in title_tokens_loweralpha]
    abstract_clean = [WDL.lemmatize(it) for it in abstract_tokens_loweralpha]
   
    return(title_clean,abstract_clean)

    
def NER(title_stem, abstract_stem):
    #print(abstract_stem)
    title_NER = (nltk.pos_tag(title_stem))
    abstract_NER = (nltk.pos_tag(abstract_stem))
    title_str = " "
    abstract_str = " "
    abstract_strr = " " 
    for key,label in title_NER:
        title_str = title_str +" " + label
    
    for key,label in abstract_NER:
        abstract_str = abstract_str + " " + label
        abstract_strr = abstract_strr + " " + key    
    #print(title_str)
    #print(abstract_str)
    #print(abstract_strr)
    return(title_str,abstract_str)
    

def cosine_sime(sentence_a, sentence_b):
    tfidf_vectorizer = TfidfVectorizer()
    tfidf_matrix = tfidf_vectorizer.fit_transform(tuple((sentence_a,sentence_b)))
    return cosine_similarity(tfidf_matrix[0:1], tfidf_matrix)[0][1]


pat = 'C:\\Users\\Devesh\\Documents\\data\\extractone\\jun-10th-dump.tar.gz.part\\keys\\key.csv'
theexcel = pd.read_csv(pat)
ultimate_dict = {'Altmetric_ID': '','Altmetric_Score':'','Cosine': ''}
cosine_list = []
df = pd.DataFrame(columns = ['Altmetric_ID', 'Altmetric_Score','Cosine'])
myFields = ['Altmetric_ID', 'Altmetric_Score', 'Cosine']
count = 0
for i in range(0,26080):
    title_token, abstract_token = split_tokenize(theexcel[['Title']].iloc[[i]].to_string(),theexcel[['Abstract']].iloc[[i]].to_string())
    title_stem,abstract_stem = token_clean(title_token[0], abstract_token[0])
    titleNER,abstractNER = NER(title_stem[2:],abstract_stem[2:])
    
    ultimate_dict['Altmetric_ID'] = theexcel['Altmetric_ID'].iloc[i]
    ultimate_dict['Altmetric_Score'] = theexcel['Altmetric_Score'].iloc[i]
    ultimate_dict['Cosine'] = (cosine_sime(titleNER,abstractNER))
    print(theexcel[['Altmetric_ID']].iloc[[i]])
    
    df.loc[-1] = ultimate_dict  # adding a row
    df.index = df.index + 1  # shifting index
    df = df.sort_index()
    df = df.dropna(axis=0, how='any')
    print(count)
    count = count +1
    #
    
    clear_output(wait=True)

    
    ultimate_dict['Altmetric_ID']= np.NaN
    ultimate_dict['Altmetric_Score']= np.NaN
    ultimate_dict['Cosine']= np.NaN

#set the path to title_abstract.csv 
PATH ='C:\\Users\\Devesh\\Documents\\data\\extractone\\jun-10th-dump.tar.gz.part\\keyt\\keytonisi.csv'
df[myFields].to_csv(PATH, index = None, header = True)
x = df['Altmetric_Score']
y = df['Cosine']
plt.scatter(y,x)

print(df.head())

## Linear Regression for cosine and Altmetric Score

In [ ]:
X = df['Altmetric_Score']
y = df['Cosine']

from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
reg = LinearRegression()
from sklearn.model_selection import cross_val_score

from scipy.stats import pearsonr

X = np.array(df['Altmetric_Score'])
y = np.array(df['Cosine'])
X = X.reshape(-1,1)
y = y.reshape(-1,1)

X_train, X_test, y_train, y_test = train_test_split(X,y, test_size = 0.3, random_state = 21)

reg.fit(X_train,y_train)
y_pred = reg.predict(X_test)


reg.fit(X_train,y_train)
y_pred = reg.predict(X_test)
plt.ylabel("Cosine Similarities of test sample")
plt.xlabel("predicted altmetric score")
plt.scatter(X_test,y_pred, color = 'black',edgecolor='blue')


print(df.shape)